# 子串

## 什么是子串？

| 概念 | 定义 | 是否连续 | 例子 |
|------|------|----------|------|
| **子串/子数组** | 原数组/字符串中**连续**的一段 | ✅ 连续 | `[1,2,3]` 的子数组：`[1]`, `[1,2]`, `[2,3]`, `[1,2,3]` |
| **子序列** | 原数组中按顺序挑选的元素，**不要求连续** | ❌ 不连续 | `[1,2,3]` 的子序列：`[1,3]`, `[2]`, `[1,2,3]` |

- **子串（Substring）**：针对字符串，连续的一段字符
- **子数组（Subarray）**：针对数组，连续的一段元素

本质上是同一个概念，只是作用对象不同。

## 常用技巧

- **滑动窗口**：维护一个动态区间，双指针移动
- **前缀和 + 哈希表**：快速计算区间和
- **单调队列/栈**：维护区间最值

---

# 560. 和为 K 的子数组

给你一个整数数组 `nums` 和一个整数 `k`，请你统计并返回该数组中 **和为 `k` 的连续子数组的个数**。

**示例 1：**
> 输入：nums = [1,1,1], k = 2  
> 输出：2

**示例 2：**
> 输入：nums = [1,2,3], k = 3  
> 输出：2

**提示：**
- `1 <= nums.length <= 2 * 10⁴`
- `-1000 <= nums[i] <= 1000`
- `-10⁷ <= k <= 10⁷`

## 方法一：前缀和 + 哈希表

**思路：**

定义前缀和 `pre[i]` 为 `nums[0..i]` 所有元素的和。那么子数组 `nums[i+1..j]` 的和就等于 `pre[j] - pre[i]`。

问题转化为：对于每个位置 `j`，有多少个位置 `i` 满足 `pre[i] == pre[j] - k`？

用哈希表记录每个前缀和出现的次数，边遍历边统计即可。

**复杂度：** 时间 O(n)，空间 O(n)

In [ ]:
from collections import defaultdict
from typing import List

class Solution:
    def subarraySum(self, nums: List[int], k: int) -> int:
        lookup = defaultdict(int)
        pre = 0
        res = 0
        for i in range(len(nums)):
            lookup[pre] += 1
            pre += nums[i]
            res += lookup[pre - k]
        return res

## 方法二：暴力枚举

**思路：**

枚举每个位置 `i` 作为子数组的结束位置，从 `i` 向左扩展，逐个累加元素，每次累加后判断当前和是否等于 `k`。

**复杂度：** 时间 O(n²)，空间 O(1)

<!-- 使用暴力法如果是python会超时 -->

In [ ]:
class Solution2:
    def subarraySum(self, nums: List[int], k: int) -> int:
        length = len(nums)
        res = 0
        for i in range(length):
            temp = nums[i]
            if temp == k:
                res += 1 
            left = i - 1
            while left >= 0:
                temp = temp + nums[left]
                if temp == k:
                    res += 1
                left -= 1
        return res

---

# 239. 滑动窗口最大值

给你一个整数数组 `nums`，有一个大小为 `k` 的滑动窗口从数组的最左侧移动到数组的最右侧。你只可以看到在滑动窗口内的 `k` 个数字。滑动窗口每次只向右移动一位。

返回 **滑动窗口中的最大值**。

**示例 1：**
> 输入：nums = [1,3,-1,-3,5,3,6,7], k = 3  
> 输出：[3,3,5,5,6,7]  
> 解释：  
> 窗口位置 &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 最大值  
> [1  3  -1] -3  5  3  6  7 &emsp; 3  
> 1 [3  -1  -3] 5  3  6  7 &emsp; 3  
> 1  3 [-1  -3  5] 3  6  7 &emsp; 5  
> 1  3  -1 [-3  5  3] 6  7 &emsp; 5  
> 1  3  -1  -3 [5  3  6] 7 &emsp; 6  
> 1  3  -1  -3  5 [3  6  7] &emsp; 7  

**示例 2：**
> 输入：nums = [1], k = 1  
> 输出：[1]

**提示：**
- `1 <= nums.length <= 10⁵`
- `-10⁴ <= nums[i] <= 10⁴`
- `1 <= k <= nums.length`

## 方法一：单调队列（双端队列）

**思路：**

维护一个单调递减的双端队列，队列中存储元素的索引。

1. 遍历数组，对于每个元素：
   - 将队尾所有小于当前元素的索引弹出（保持单调递减）
   - 将当前索引加入队尾
   - 如果队首索引已超出窗口范围，弹出队首
   - 当窗口形成后（`i >= k-1`），队首元素即为当前窗口最大值

**为什么有效：** 队列保持单调递减，队首始终是当前窗口的最大值。小于当前元素的值不可能成为后续窗口的最大值，所以可以直接弹出。

**复杂度：** 时间 O(n)，空间 O(k)

In [ ]:
from collections import deque
from typing import List

class Solution:
    def maxSlidingWindow(self, nums: List[int], k: int) -> List[int]:
        q = deque()
        res = []
        length = len(nums)
        for i in range(length):
            while q and nums[i] >= nums[q[-1]]:
                q.pop()
            q.append(i)
            if q[0] < i - k + 1:
                q.popleft()
            if i - k + 1 >= 0:
                res.append(nums[q[0]])
        return res

## 方法二：暴力枚举

**思路：**

直接遍历每个窗口位置，对每个窗口使用 `max()` 函数求最大值。

特判：当 `k == 1` 时，直接返回原数组的拷贝。

**关键点解释：**

- `len - k + 1`：窗口总数。最后一个窗口起始索引为 `n - k`，所以共有 `n - k + 1` 个窗口
- `nums[i : i + k]`：从索引 `i` 开始取 `k` 个元素（Python 切片左闭右开）

```
数组: [1, 3, -1, -3, 5, 3, 6, 7]   n = 8, k = 3
       ↑-------↑                   窗口0: nums[0:3]
          ↑-------↑                窗口1: nums[1:4]
             ↑-------↑             窗口2: nums[2:5]
                ↑-------↑          窗口3: nums[3:6]
                   ↑-------↑       窗口4: nums[4:7]
                      ↑-------↑    窗口5: nums[5:8]
                      
共 8 - 3 + 1 = 6 个窗口
```

**复杂度：** 时间 O(nk)，空间 O(1)

<!-- Python 暴力法会超时 -->

In [ ]:
class Solution2:
    def maxSlidingWindow(self, nums: List[int], k: int) -> List[int]:
        res = []
        if k == 1:
            res = nums[:]
        else:
            for i in range(len(nums) - k + 1):
                temp = max(nums[i : i + k])
                res.append(temp)
        return res